In [1]:

import MeCab
import pandas as pd
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:

def is_kanji(char):
   
    return 'CJK UNIFIED' in unicodedata.name(char)
def load_data():
    data= pd.read_csv("Kanji_char (1).csv")
    df = data.set_index('new')['strokes'].to_dict()
    return df
    
    
def calculate_difficulty_score(japanese_text):
    # Tokenizing the Japanese text
    tokenized_words = tokenize_japanese_text(japanese_text)

    #Loading csv file of Kanji character with strokes mentioned and converting it into python dictionary for key-value pair
    kanji_strokes_data = load_data()

    #threshold for the top 5% of strokes
    top_5_percent_threshold = calculate_top_percent_threshold(kanji_strokes_data, 5)

    difficulty_score = 0 #initial score
    counter =0

    for word in tokenized_words:
        for char in word:
            if is_kanji(char):
                counter+=1
                strokes = get_number_of_strokes(char, kanji_strokes_data)

                if strokes > top_5_percent_threshold:
                    difficulty_score += 1
    print("kanji Character count:" , counter)

    return difficulty_score

def tokenize_japanese_text(text):
    mecab = MeCab.Tagger("-Owakati")
    tokenized_words = mecab.parse(text).split()
    return tokenized_words

def calculate_top_percent_threshold(strokes_data, top_percent):
    all_strokes = list(strokes_data.values())
    all_strokes.sort(reverse=True)
    threshold_index = int(len(all_strokes) * (top_percent / 100.0))
    top_percent_threshold = all_strokes[threshold_index]
    return top_percent_threshold

def get_number_of_strokes(kanji_char, strokes_data):
    return strokes_data.get(kanji_char, 0)




In [3]:
# Specify the file path
file_path = 'testCases.txt'

# Open the file with the appropriate encoding (UTF-8)
with open(file_path, 'r', encoding='utf-8') as file:
    # Read the content of the file
    japanese_text = file.read()

# Example usage
# kanji_text = "新しい言葉や難解な漢字を学ぶことは言語学習の一部です。"
# kanji_text2 = "よし子さんは、二月二十日から一 週しゅう間かん、北ほっ海かい道どうに行きました。一日目は、つかれて何なにもできませんでしたが、二日目からは、スキーをしたり、温おんせんに入はいったり、カラオケに行いったりして、とても楽たのしかったです。来らい年ねんの冬ふゆは、スノーボードを習ならいたいです。"
# kanji_text3 = "こんにちは。お元気ですか。夏休みは何をしていますか。私はこの前、オリンピックを見に行きました。すごく良よかったです！"
# kanji_text4 ="毎日新聞社が発行する週刊点字新聞「点字毎日」は2022年、創刊100年を迎えた。"
# kanji_text5 = "新聞の文化的使命を徹底せしめんとするにほかありません。かくして、一方には盲人に対し、一個の独立せる市民として社会に活動するに必要な知識と勇気と慰安とを与え、他方には、これまで盲人に対して眠れる社会の良心を呼び覚まさんとするにあります」と、その理念を格調高くうたい上げている。"
difficulty_score = calculate_difficulty_score(japanese_text)

print("Difficulty Score:", difficulty_score)

kanji Character count: 59
Difficulty Score: 1


In [6]:
font_family = "Noto Sans JP"
plt.rcParams["font.family"] = font_family

def plot_strokes_distribution(strokes_data):
    plt.bar(strokes_data.keys(), strokes_data.values())
    plt.xlabel('Kanji Characters')
    plt.ylabel('Number of Strokes')
    plt.title('Strokes Distribution in Kanji Characters')
    plt.show()

def plot_difficulty_pie_chart(difficulty_stats):
    labels = ['Difficult Words', 'Easy Words']
    sizes = [difficulty_stats['difficult'], difficulty_stats['easy']]
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    plt.title('Difficulty Distribution in Japanese Text')
    plt.show()
    
